In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
df = pd.read_csv("../data/train/variants/fr/variants_hist_data.csv", sep =';')
df2 = pd.read_csv("../data/train/all_data_merged/fr/Enriched_Covid_history_data.csv")
df2["time"] = pd.to_datetime(df2["time"])
df['dep'] = df['dep'].replace({'2A':'201','2B':'202'}).astype(int)
df = df[df['dep']<203]
#df = df.groupby(["dep","semaine"], level = -1).sum()["dep","semaine","Nb_susp_501Y_V1","Nb_susp_501Y_V2_3"]
df= df.groupby(['dep', 'semaine'])[["dep","semaine","Nb_susp_501Y_V1","Nb_susp_501Y_V2_3"]].sum().drop(columns = ["dep"]).reset_index()
df

,dep,semaine,Nb_susp_501Y_V1,Nb_susp_501Y_V2_3
0,1,2021-02-12-2021-02-18,686,42
1,1,2021-02-13-2021-02-19,706,46
2,1,2021-02-14-2021-02-20,730,46
3,1,2021-02-15-2021-02-21,748,48
4,1,2021-02-16-2021-02-22,852,52
...,...,...,...,...
4219,202,2021-03-23-2021-03-29,302,14
4220,202,2021-03-24-2021-03-30,302,18
4221,202,2021-03-25-2021-03-31,328,18
4222,202,2021-03-26-2021-04-01,318,16


In [3]:
#10 premiers caractères datetime
#réaffecter directement
#récupérer la date minimum de variant_list (=df avec datetime)

def to_datalist(row):
    date = pd.to_datetime(row["semaine"][0:10], yearfirst = True)
    return date

df['jour'] = df.apply(to_datalist, axis=1)
df.drop(columns='semaine', inplace=True)
df.rename(columns={'jour':'semaine'}, inplace=True)

In [4]:
def create_possibilities(row):
    return pd.date_range(start = row['semaine'], periods=7).tolist()

df['7_days'] = df.apply(create_possibilities, axis=1)
df

,dep,Nb_susp_501Y_V1,Nb_susp_501Y_V2_3,semaine,7_days
0,1,686,42,2021-02-12,"[2021-02-12 00:00:00, 2021-02-13 00:00:00, 202..."
1,1,706,46,2021-02-13,"[2021-02-13 00:00:00, 2021-02-14 00:00:00, 202..."
2,1,730,46,2021-02-14,"[2021-02-14 00:00:00, 2021-02-15 00:00:00, 202..."
3,1,748,48,2021-02-15,"[2021-02-15 00:00:00, 2021-02-16 00:00:00, 202..."
4,1,852,52,2021-02-16,"[2021-02-16 00:00:00, 2021-02-17 00:00:00, 202..."
...,...,...,...,...,...
4219,202,302,14,2021-03-23,"[2021-03-23 00:00:00, 2021-03-24 00:00:00, 202..."
4220,202,302,18,2021-03-24,"[2021-03-24 00:00:00, 2021-03-25 00:00:00, 202..."
4221,202,328,18,2021-03-25,"[2021-03-25 00:00:00, 2021-03-26 00:00:00, 202..."
4222,202,318,16,2021-03-26,"[2021-03-26 00:00:00, 2021-03-27 00:00:00, 202..."


In [5]:
referencedate = df['semaine'].min()

In [6]:
# regarder pour chaque row de Enriched si :
# - le jour est avant le jour de début des variants
# - le jour est dans la semaine suivant le row des variants
# - append à Enriched+variant (= df de enriched + infos sur les variants)

In [7]:
def check_variant(v_row, date):
    if date in v_row['7_days']:
        return v_row['Nb_susp_501Y_V1'], v_row['Nb_susp_501Y_V2_3']
  

In [8]:
def enriched_variant(row):
    date = row['time']
    depnum = row['numero']
    if date < referencedate:
        row['Nb_susp_501Y_V1'], row['Nb_susp_501Y_V2_3'] = 0, 0
    else : 
        df_dep = df[df['dep']==depnum]
        ans=[]
        res = df_dep.apply(check_variant,date=date, axis=1)
        ans.append(next((el for el in res if el is not None), None)) #get the first non null element of res
        row['Nb_susp_501Y_V1'], row['Nb_susp_501Y_V2_3'] = ans[0][0], ans[0][1] 
    return None

In [9]:
df2.apply(enriched_variant, axis=1)

0        None
1        None
2        None
3        None
4        None
         ... 
30585    None
30586    None
30587    None
30588    None
30589    None
Length: 30590, dtype: object

In [10]:
df2[['time', 'Nb_susp_501Y_V1', 'Nb_susp_501Y_V2_3']]

,time,Nb_susp_501Y_V1,Nb_susp_501Y_V2_3
0,2020-05-14,0,0
1,2020-05-15,0,0
2,2020-05-16,0,0
3,2020-05-17,0,0
4,2020-05-18,0,0
...,...,...,...
30585,2021-03-27,9561,432
30586,2021-03-28,9557,428
30587,2021-03-29,9496,396
30588,2021-03-30,8643,364
